# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def point_dist(x1, y1, x2, y2):
    """
    This function finds the distance between two pixels
    """
    return math.sqrt(float((x1 - x2) ** 2 + (y1 - y2) ** 2))

def segment_slope(x1, y1, x2, y2):
    """
    This function finds the slope of a line segment between two pixels
    """
    return (float(y1-y2)/(float(x1-x2) if x1 != x2 else 0.001))

# Globals used to track previous frame's lane lines
llane = (0, 0, 0, 0)
rlane = (0, 0, 0, 0)
slope_l = 0.0
slope_r = 0.0
int_l = 0.0
int_r = 0.0

def init_frames():
    """
    This function reinitializes the globals that keep track of lanes between frames in a video.
    It should be called before using `hough_lines` or `draw_lines` on an image or on the first frame
    of a new video.
    """
    global llane, rlane, slope_l, slope_r, int_l, int_r
    llane = (0, 0, 0, 0)
    rlane = (0, 0, 0, 0)
    slope_l = 0.0
    slope_r = 0.0
    int_l = 0.0
    int_r = 0.0

def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    global llane, rlane, slope_l, slope_r, int_l, int_r
    
    # Horizontal center of screen
    rc_x_c = img.shape[1]/2
    # Find left and right reference pixel coordinates for approximate location of where the lanes should be
    # If they weren't set before, initialize to sane initial values
    if (rlane == (0,0,0,0)):
        rc_x_l = img.shape[1] / 3
        rc_x_r = rc_x_l * 2
        rc_y_l = img.shape[0]
        rc_y_r = img.shape[0]
    else: # Otherwise, get them from the lane found in the last frame, which reduces noise
        rc_x_l = (llane[0] + llane[2])/2
        rc_x_r = (rlane[0] + rlane[2])/2
        rc_y_l = (llane[1] + llane[3])/2
        rc_y_r = (rlane[1] + rlane[3])/2
    
    # Variables to keep track of state inside loop
    ldist = point_dist(0, 0, img.shape[0], img.shape[1])
    rdist = point_dist(0, 0, img.shape[0], img.shape[1])
    lslope = 0.0
    rslope = 0.0
    lint = 0.0
    rint = 0.0
    # Find lane ends closest to car by finding the lines which come closest to left and right reference pixels
    for line in lines:
        for x1, y1, x2, y2 in line:
            slope = segment_slope(x1, y1, x2, y2)
            intercept = y1 - (slope * x1)
            if ((min(x1, x2) < rc_x_c) and (slope < -0.25)): # On the left and negative (in image coordinates) slope
                dist1 = point_dist(x1, y1, rc_x_l, rc_y_l)
                dist2 = point_dist(x2, y2, rc_x_l, rc_y_l)
                dist3 = point_dist((x1+x2)/2, (y1+y2)/2, rc_x_l, rc_y_l)
                if ((min(dist1, dist2, dist3) < ldist) and ((lslope == 0) or (slope_l == 0))) or \
                    ((slope_l != 0) and (abs(slope - slope_l) < abs(lslope - slope_l)) and \
                        (abs(intercept - int_l) < abs(lint - int_l))):
                    ldist = min(dist1, dist2, dist3)
                    llane = (x1, y1, x2, y2)
                    lslope = slope
                    lint = intercept
            if (max(x1, x2) > rc_x_c) and (slope > 0.25): # On the right and positive (in image coordinates) slope
                dist1 = point_dist(x1, y1, rc_x_r, rc_y_r)
                dist2 = point_dist(x2, y2, rc_x_r, rc_y_r)
                dist3 = point_dist((x1+x2)/2, (y1+y2)/2, rc_x_r, rc_y_r)
                if ((min(dist1, dist2, dist3) < rdist) and ((rslope == 0) or (slope_r == 0))) or \
                    ((slope_r != 0) and (abs(slope - slope_r) < abs(rslope - slope_r)) and \
                        (abs(intercept - int_r) < abs(rint - int_r))):
                    rdist = min(dist1, dist2, dist3)
                    rlane = (x1, y1, x2, y2)
                    rslope = slope
                    rint = intercept
    
    slope_l = lslope
    slope_r = rslope
    int_l = lint
    int_r = rint
    
    # Extend and draw the lane lines we found, from bottom of image to their intersection at the "horizon" (not perfect)
    ly1 = img.shape[1]
    lx1 = int((ly1 - int_l) / slope_l)
    ry1 = ly1
    rx1 = int((ry1 - int_r) / slope_r)
    x2 = int((int_l - int_r) / (slope_r - slope_l))
    y2 = int((slope_l * x2) + int_l)
    cv2.line(img, (lx1, ly1), (x2, y2), color, thickness)
    cv2.line(img, (rx1, ry1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    # Don't use splat; doesn't work in Python 2.7
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, a=0.8, b=1., l=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, a, img, b, l)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
# Count number of figures to display multiple images at once
i = 0
# Go through each image in test_images subdirectory
for img_file in os.listdir("test_images/"):
    # Initialize globals so the last image's lane lines aren't used in calculations
    init_frames()
    # Read the image
    img = mpimg.imread('test_images/' + img_file)
    # Set figure number
    i += 1
    plt.figure(i)
    # Make image grayscale for processing based on brightness gradient
    gray_img = grayscale(img)
    # Add Gaussian blur to prevent finding too many edges
    blur_img = gaussian_blur(gray_img, 3)
    # Use Canny edge detection to find edges
    canny_img = canny(blur_img, 50, 150)
    # Select just the region we're LIKELY to be interested in
    vertices = np.array([[
                (0, img.shape[0]),
                (img.shape[1] * .4, img.shape[0] * .4),
                (img.shape[1] * .6, img.shape[0] * .4),
                (img.shape[1], img.shape[0])
            ]], dtype=np.int32)
    clipped_img = region_of_interest(canny_img, vertices)
    # Use Hough transform to find lines
    hough_img = hough_lines(clipped_img, 2, np.pi/180, 25, 40, 20)
    # Plot image
    plt.imshow(weighted_img(hough_img, img))
plt.show()

run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # Make image grayscale for processing based on brightness gradient
    gray_img = grayscale(image)
    # Add Gaussian blur to prevent finding too many edges
    blur_img = gaussian_blur(gray_img, 3)
    # Use Canny edge detection to find edges
    canny_img = canny(gray_img, 50, 150)
    # Select just the region we're LIKELY to be interested in
    vertices = np.array([[
                (0, image.shape[0]),
                (image.shape[1] * .4, image.shape[0] * .4),
                (image.shape[1] * .6, image.shape[0] * .4),
                (image.shape[1], image.shape[0])
            ]], dtype=np.int32)
    clipped_img = region_of_interest(canny_img, vertices)
    # Use Hough transform to find lines
    hough_img = hough_lines(clipped_img, 2, np.pi/180, 25, 40, 20)
    # Return weighted image
    return weighted_img(hough_img, image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
init_frames()
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
init_frames()
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

### Alex's Thoughts

There are quite a few things I can think of to make my algorithm better.

* Finding edges/Hough lines in lower contrast settings, such as when the car goes over the bridge in the challenge video. In the time I had, I couldn't get Hough lines to be found for the yellow line on the left on that bridge; the closest I could get was the bottom of the Jersey wall.

* A better heuristic for finding a lane line starting point. I went off the position and slope based on the camera, but this wouldn't work very well in the case of switching lanes or turning.

* Combining multiple Hough lines to create a lane line instead of taking the closest one and extending it to the bottom of the screen and the "intersection" of the lane lines. This is especially evident when the road curves.

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
init_frames()
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))